In [2]:
import json
import os
import math
import copy # Para crear copias de los diccionarios si es necesario

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)
TARGET_SCENARIO_ID = "scenario_1" # Escenario específico a analizar

# --- Ruta del archivo ---
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json"

print(f"Archivo de entrada: {input_file}")
print(f"Analizando SOLAMENTE para: '{TARGET_SCENARIO_ID}'")

# Listas para clasificar los pares del escenario objetivo
coordinated_pairs = []
uncoordinated_pairs = []

try:
    # 1. Cargar el archivo JSON (lista de pares)
    print("Cargando datos (lista de pares)...")
    with open(input_file, 'r') as f:
        relay_pairs_data = json.load(f)
    print("Datos cargados correctamente.")

    # Validar que es una lista
    if not isinstance(relay_pairs_data, list):
        raise TypeError(f"Error: El archivo {input_file} no contiene una lista JSON.")

    # 2. Procesar cada par de relés, filtrando por scenario_id
    print(f"Calculando delta_t, mt y clasificando pares para '{TARGET_SCENARIO_ID}'...")
    total_pairs_read = 0
    scenario_pairs_found = 0
    skipped_pairs_count = 0 # Pares omitidos DENTRO del scenario_id objetivo

    for pair_entry in relay_pairs_data:
        total_pairs_read += 1
        if not isinstance(pair_entry, dict):
            # Contar como leído pero no procesado si no es diccionario
            continue

        # --- FILTRAR POR SCENARIO_ID ---
        current_scenario_id = pair_entry.get("scenario_id")
        if current_scenario_id != TARGET_SCENARIO_ID:
            continue # Omitir este par si no es del escenario objetivo
        # ---------------------------------

        # Si llegamos aquí, el par pertenece a TARGET_SCENARIO_ID
        scenario_pairs_found += 1

        # Obtener información de los relés de forma segura
        main_relay_info = pair_entry.get('main_relay')
        backup_relay_info = pair_entry.get('backup_relay')

        if not isinstance(main_relay_info, dict) or not isinstance(backup_relay_info, dict):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Falta información de relé principal o de respaldo en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')}. Par omitido.")
            skipped_pairs_count += 1
            continue

        # Obtener tiempos de operación de forma segura
        main_time = main_relay_info.get('Time_out')
        backup_time = backup_relay_info.get('Time_out')

        # Validar que los tiempos son números
        if not isinstance(main_time, (int, float)) or not isinstance(backup_time, (int, float)):
            print(f"Advertencia ({TARGET_SCENARIO_ID}): Tiempo(s) de operación no numéricos o faltantes en: {pair_entry.get('line', 'N/A')}-{pair_entry.get('fault', 'N/A')} (Main: {main_time}, Backup: {backup_time}). Par omitido.")
            skipped_pairs_count += 1
            continue

        # --- Realizar Cálculos ---
        delta_t = backup_time - main_time - CTI
        mt = (delta_t - abs(delta_t)) / 2  # Penalización solo si delta_t es negativo

        # Crear una copia del par y añadirle los resultados del cálculo
        pair_info = copy.deepcopy(pair_entry)
        pair_info['delta_t'] = delta_t
        pair_info['mt'] = mt

        # --- Clasificar (solo pares del TARGET_SCENARIO_ID) ---
        if delta_t >= 0:
            coordinated_pairs.append(pair_info)
        else:
            uncoordinated_pairs.append(pair_info)

    print("Procesamiento de pares completado.")

    # 3. Calcular Métricas Finales (solo para TARGET_SCENARIO_ID)
    if scenario_pairs_found == 0:
         print (f"No se encontraron pares válidos para '{TARGET_SCENARIO_ID}' en el archivo.")
    else:
        total_valid_pairs_scenario = len(coordinated_pairs) + len(uncoordinated_pairs) # Pares válidos DENTRO del escenario
        miscoordination_count_scenario = len(uncoordinated_pairs)
        # Sumar 'mt' solo de los pares del escenario objetivo (que son los únicos en las listas)
        tmt_total_scenario = sum(pair["mt"] for pair in coordinated_pairs + uncoordinated_pairs)


        # 4. Imprimir Resultados
        print(f"\n--- Resultados del Análisis de Coordinación para '{TARGET_SCENARIO_ID}' ---")
        print(f"Total de pares leídos del archivo: {total_pairs_read}")
        print(f"Total de pares encontrados para '{TARGET_SCENARIO_ID}': {scenario_pairs_found}")
        if skipped_pairs_count > 0:
            print(f"Pares omitidos DENTRO de '{TARGET_SCENARIO_ID}' (datos inválidos/faltantes): {skipped_pairs_count}")
        print(f"Total de pares válidos analizados para '{TARGET_SCENARIO_ID}': {total_valid_pairs_scenario}")
        print(f"Número de pares coordinados (delta_t >= 0) en '{TARGET_SCENARIO_ID}': {len(coordinated_pairs)}")
        print(f"Número de pares DESCOORDINADOS (delta_t < 0) en '{TARGET_SCENARIO_ID}': {miscoordination_count_scenario}")
        print(f"Suma total de penalización por descoordinación (TMT Total) para '{TARGET_SCENARIO_ID}': {tmt_total_scenario:.5f}") # Imprimir con 5 decimales

except FileNotFoundError:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
except TypeError as e:
    print(f"Error CRÍTICO: Problema con el tipo de datos esperado en el archivo JSON: {e}")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")

Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json
Analizando SOLAMENTE para: 'scenario_1'
Cargando datos (lista de pares)...
Datos cargados correctamente.
Calculando delta_t, mt y clasificando pares para 'scenario_1'...
Procesamiento de pares completado.

--- Resultados del Análisis de Coordinación para 'scenario_1' ---
Total de pares leídos del archivo: 6800
Total de pares encontrados para 'scenario_1': 100
Total de pares válidos analizados para 'scenario_1': 100
Número de pares coordinados (delta_t >= 0) en 'scenario_1': 55
Número de pares DESCOORDINADOS (delta_t < 0) en 'scenario_1': 45
Suma total de penalización por descoordinación (TMT Total) para 'scenario_1': -28.06440
